In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# 1. Verify we are using your Quadro M1200
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Hardware check: Running on {device.upper()}\n")

# 2. Download and load the FLAN model and tokenizer
model_name = "google/flan-t5-small"
print(f"Loading {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# 3. Give it a test prompt
prompt = "Translate to German: I am successfully running AI on my GPU!"
print(f"\nPrompt: {prompt}")

# 4. Generate the response
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Response: {response}")

Hardware check: Running on CUDA

Loading google/flan-t5-small...


C:\Users\Nil\anaconda3\envs\transformers-env\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Nil\anaconda3\envs\transformers-env\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Prompt: Translate to German: I am successfully running AI on my GPU!


C:\Users\Nil\anaconda3\envs\transformers-env\lib\site-packages\transformers\generation\utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Response: Ich habe erfolgreiches AI auf meinem GPU!


In [3]:
from datasets import load_dataset
dataset = load_dataset("knkarthick/dialogsum")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})


In [4]:
print(f"{dataset['test'][40]['dialogue']}")

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.


In [5]:
# check the system and load the dataset
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"The hardware to be used to run the model is: {device.upper()}\n")

The hardware to be used to run the model is: CUDA



In [6]:
# download the model and tokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

C:\Users\Nil\anaconda3\envs\transformers-env\lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
# testing the model
prompt = "Translate to Hindi: I am running large language model on my system"
print(f"\nPrompt given to the model: {prompt}")
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"The response of model : {response}")


Prompt given to the model: Translate to Hindi: I am running large language model on my system
The response of model :          


In [16]:
dialogue_text = dataset['test'][40]['dialogue']
prompt = f"Give me the sentiment for {dialogue_text}"
print(f"The input to my model: {prompt}\n")
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"The sentiment analysis by the model for the conversation is: {response}")

The input to my model: Give me the sentiment for #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

The sentiment analysis by the model for the conversation is: negative


In [20]:
dialogue_text = dataset['test'][40]['dialogue']
human_summary = dataset['test'][40]['summary']
print(f"The human written summary for the input to LLM is: {human_summary}")
prompt = f"Summarize the coversation: {dialogue_text}"
inputs = tokenizer(prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=50)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"The analysis of the model is: {response}")

The human written answer for the input to LLM is: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
The analysis of the model is: It's ten to nine by my watch.


In [23]:
example_dialogue = dataset['test'][200]['dialogue'] 
example_summary = dataset['test'][200]['summary']
target_dialogue = dataset['test'][40]['dialogue']
print(f"The example dialogue: {example_dialogue}")
print(f"The human summary: {example_summary}")
one_shot_prompt = f"""Summarize this dialogue:
{example_dialogue}
Summary: {example_summary}

Summarize this dialogue:
{target_dialogue}
Summary:"""
# passing to the model
inputs = tokenizer(one_shot_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens = 100)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"--- One-Shot Model Summary ---\n{response}")

The example dialogue: #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
The human summary: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
--- One-Shot Model Summary ---
It's ten to nine by my watch.
